In [ ]:
'''Import statements and settings'''
%matplotlib inline

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms
import matplotlib.gridspec as gridspec

import scipy
import scipy.constants
import scipy.stats as st
from scipy.stats import norm
from scipy.stats import shapiro

import random

import data_util as util


In [4]:
'''Pull in data -- standardized'''


# Set root directory
sim_type = "ebind"  # ebind, lig_num, or s
sim_num = 8  # binding energy (2-20), ligand number (100-1100), or solvent parameter s (0.1-1.0)
sim_len = 25  # sim_len million timesteps
short = True

if short:
    root = "sims_out/%s/short/%s%d/" % (sim_type, sim_type, sim_num)
else:
    # FOR extended roots only, you must manually enter the sim length
    root = "sims_out/%s/extended/%s%d_%dmil" % (sim_type, sim_type, sim_len)


# Pull in coords.dmp and log.lammps
first_tstep, last_tstep, tstep_size = 10000, sim_len*1000000+10000, 5000  # params same for coords and thermo
# coords.dmp
dmp = root + "coords.dmp"
coords_dat = "results/sims/%s/out/%s%d/coords.dat" % (sim_type, sim_type, sim_num)
tot_lns, lns_per_time, bad_lns, tsteps_on = sim_len*2*1000+11, 10, 9, True
coords_cols = {
    0: "c_coords"
}
coords_data = util.dmp_to_dat(dmp, coords_dat, tot_lns, lns_per_time, bad_lns, coords_cols, tsteps_on=tsteps_on, first_tstep=
                         first_tstep, last_tstep=last_tstep, tstep_size=tstep_size)
# log.lammps
thermo = root + "log.lammps"
thermo_dat = "results/sims/%s/out/%s%d/thermo.dat" % (sim_type, sim_type, sim_num)
header_len, last_ln, tot_len = 246, sim_len*2*100+266 - 32, sim_len*2*100+266
thermo_cols = {  
    1: "Temp",  
    2: "E_pair",  
    3: "E_mol",  
    4: "TotEng",  
    #5: "Press"
}
thermo_data = util.thermo_to_dat(thermo, thermo_dat, header_len, last_ln, tot_len, thermo_cols, first_tstep, last_tstep, 
                                 tstep_size)

#Hacky -- lazy, so just converting these to dictionaries since everything was written for this
# Convert to dictionary of numpy arrays:
thermo_data = thermo_data.to_dict("series")
for key in thermo_data:
    thermo_data[key] = np.array(thermo_data[key].values)
coords_data = coords_data.to_dict("series")
for key in coords_data:
    coords_data[key] = np.array(coords_data[key].values)

# Reduce size
reduce_size = False
div = 8
if reduce_size:
    for key in thermo_data:
        og_array = thermo_data[key]
        new_list = []
        for idx, element in enumerate(og_array):
                if idx % div == 0:
                    new_list.append(element)

        thermo_data[key] = np.array(new_list)
    for key in coords_data:
        og_array = coords_data[key]
        new_list = []
        for idx, element in enumerate(og_array):
                if idx % div == 0:
                    new_list.append(element)

        coords_data[key] = np.array(new_list)

print("Number of timesteps: ", len(thermo_data["timestep"]))


NameError: name 'util' is not defined